In [25]:
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc

In [26]:
from pyomo.environ import *
import numpy as np
import pandas as pd

In [27]:
df1 = pd.read_csv('lab5_ex1.csv',index_col=0,header=0)


\begin{equation}
\text{Let C denote the following matrix} \\
   \begin{matrix}    
627&617&1270&2072&1415&2943&1064&4265\\
159&1432&351&3372&237&3373&1456&5059\\
609&1943&77&3365&298&3021&1484&4803\\
2143&671&3876&537&1902&1941&1682&3348\\
2742&2089&1931&2199&1226&739&691&1128\\
2053&860&2995&68&2069&2065&993&3342\\
656&817&1245&2141&1534&3351&1137&3887\\
195&1862&1331&2921&402&2825&563&4441\\
 \end{matrix} 
\end{equation}

Let $i$ take values from the set $\{1,2,3,4,5,6,7,8\}$ corresponding to Ahmedabad, Bengaluru, Chennai, Delhi, Kolkata, Lucknow,Mumbai,Hyderabad respevtively. 

Let $j$ take values from the set $\{1,2,3,4,5,6,7,8\}$ corresponding to Hassan, Jodhpur, Trichy, Simla, Vijayawada, Dibrugarh,Raipur,Agartala respectively. 

Let $x_{ij}$ denote the number of units transported from warehouse $i$ to market $j$.

Let $c_{ij}$ denote the element in the $i^{th}$ row and $j^{th}$ column of C.

## Model:

Decision variables:\
$x_{ij} ,\qquad$ $i= 1,2,3,4,5,6,7,8 \qquad$ $j = 1,2,3,4,5,6,7,8$.

Objective: $\text{minimize} \sum_{i=1}^{8} \sum_{j=1}^{8} c_{ij} x_{ij} $

Subject to:

\begin{align}
\sum_{j=1}^{8} x_{1j} &\leq 185 \qquad  \text{supply at Ahmedabad } \nonumber \\
\sum_{j=1}^{8} x_{2j} &\leq 350  \qquad  \text{supply at Bengaluru } \nonumber \\
\sum_{j=1}^{8} x_{3j} &\leq 405 \qquad   \text{supply at Chennai} \nonumber \\
\sum_{j=1}^{8} x_{4j} &\leq 280 \qquad   \text{supply at Delhi } \nonumber \\
\sum_{j=1}^{8} x_{5j} &\leq 105  \qquad  \text{supply at Kolkata } \nonumber \\
\sum_{j=1}^{8} x_{6j} &\leq 75 \qquad   \text{supply at Lucknow } \nonumber \\
\sum_{j=1}^{8} x_{7j} &\leq 425  \qquad  \text{supply at Mumbai } \nonumber \\
\sum_{j=1}^{8} x_{8j} &\leq 270  \qquad  \text{supply at Hyderabad } \nonumber \\
\sum_{i=1}^{8} x_{i1} &= 175 \qquad \text{ demand at Hassan } \nonumber \\
\sum_{i=1}^{8} x_{i2} &= 400 \qquad  \text{ demand at Jodhpur } \nonumber \\
\sum_{i=1}^{8} x_{i3} &= 550  \qquad \text{ demand at Trichy } \nonumber \\
\sum_{i=1}^{8} x_{i4} &= 70 \qquad  \text{ demand at Simla } \nonumber \\
\sum_{i=1}^{8} x_{i5} &= 650 \qquad  \text{demand at Vijaywada } \nonumber \\
\sum_{i=1}^{8} x_{i6} &= 100 \qquad \text{demand at Dibrugarh } \nonumber \\
\sum_{i=1}^{8} x_{i7} &= 65 \qquad \text{demand at Raipur } \nonumber \\
\sum_{i=1}^{8} x_{i8} &= 85 \qquad  \text{demand at Agartala } \nonumber \\
x_{ij} &\geq 0 , x_{ij} \in \mathbb{Z} \quad \forall i \in \{1,2,3,4,5,6,7,8\}, \ \forall j \in \{1,2,3,4,5,6,7,8\} 
\end{align}\



In [28]:

n = len(df1.columns)-1
m = len(df1)-1

rows = np.arange(m)
cols = np.arange(n)

In [29]:
costs = df1.values[:-1,:-1]
sup_coefs =df1.iloc[:-1,-1]
dem_coefs = df1.iloc[-1,:-1]

In [30]:
m1 = ConcreteModel()
m1.x = Var(rows,cols,domain=NonNegativeIntegers)
m1.cons = ConstraintList()


In [31]:
#Supply constraints
for i in rows:
    m1.cons.add(sum(m1.x[i,j] for j in cols)<=sup_coefs[i])

#demand constraints
for j in cols:
    m1.cons.add(sum(m1.x[i,j] for i in rows )==dem_coefs[j])

In [32]:
m1.cost = Objective(expr=summation(costs,m1.x))

In [33]:
solver = SolverFactory('cbc')
opt1 = solver.solve(m1)


### 7)

In [34]:
for i in rows:
  for j in cols:
    num = m1.x[i,j]()
    if num != 0:
      print('Number of units transported from {} to {} is {}'.format(df1.index[i],df1.columns[j],num))

Number of units transported from Ahmedabad to Jodhpur is 185.0
Number of units transported from Bengaluru to Vijayawada is 350.0
Number of units transported from Chennai to Trichy is 375.0
Number of units transported from Chennai to Vijayawada is 30.0
Number of units transported from Delhi to Jodhpur is 200.0
Number of units transported from Delhi to Dibrugarh is 80.0
Number of units transported from Kolkata to Dibrugarh is 20.0
Number of units transported from Kolkata to Agartala is 85.0
Number of units transported from Lucknow to Simla is 70.0
Number of units transported from Lucknow to Raipur is 5.0
Number of units transported from Mumbai to Hassan is 175.0
Number of units transported from Mumbai to Jodhpur is 15.0
Number of units transported from Mumbai to Trichy is 175.0
Number of units transported from Mumbai to Raipur is 60.0
Number of units transported from Hyderabad to Vijayawada is 270.0


In [35]:
print('Optimal total cost of transportation is',m1.cost())

Optimal total cost of transportation is 1166465.0


### 8)

In [36]:
#Instead of changing the csv file and loading it into a new dataframe, we can skip a step and directly make the change in the dataframe

df2=df1.copy() # creating a copy to not disturb original dataframe

df2.loc['Bengaluru','Hassan'] = 999999 
#we make the cost of using Bangalore hassan route very high, so that using this route will drastically increase cost and this would not be part of an optimal solution 


In [37]:
m2=m1.clone() # Using the same model as before

m2.cost.deactivate()
m2.cost2 = Objective(expr=summation(df2.values[:-1,:-1],m2.x))
#Updating objective to consider the change that we made 

In [38]:
opt2 = solver.solve(m2)

### 9)

In [39]:
for i in rows:
  for j in cols:
    num2 = m2.x[i,j]()
    if num2 != 0:
      print('Number of units transported from {} to {} is {}'.format(df2.index[i],df2.columns[j],num2))

Number of units transported from Ahmedabad to Jodhpur is 185.0
Number of units transported from Bengaluru to Vijayawada is 350.0
Number of units transported from Chennai to Trichy is 375.0
Number of units transported from Chennai to Vijayawada is 30.0
Number of units transported from Delhi to Jodhpur is 200.0
Number of units transported from Delhi to Dibrugarh is 80.0
Number of units transported from Kolkata to Dibrugarh is 20.0
Number of units transported from Kolkata to Agartala is 85.0
Number of units transported from Lucknow to Simla is 70.0
Number of units transported from Lucknow to Raipur is 5.0
Number of units transported from Mumbai to Hassan is 175.0
Number of units transported from Mumbai to Jodhpur is 15.0
Number of units transported from Mumbai to Trichy is 175.0
Number of units transported from Mumbai to Raipur is 60.0
Number of units transported from Hyderabad to Vijayawada is 270.0


In [40]:
print('Optimal total cost of transportation is',m2.cost())

Optimal total cost of transportation is 1166465.0



### 10)

In [41]:
#Instead of changing the csv file and loading it into a new dataframe, we can skip a step and directly make the change in the dataframe


df3 = df2.copy() #Copying from df2 where bangalore hassan route constraint is already taken care of

df3.loc['Lucknow','Simla'] = 999999
df3.loc['Kolkata','Agartala'] = 999999
#we make the cost of using the restricted routes very high, so that using this route will drastically increase cost and these would not be part of an optimal solution 

In [42]:
m3=m1.clone() #Using same model as original problem

m3.cost.deactivate()
m3.cost3 = Objective(expr=summation(df3.values[:-1,:-1],m3.x)) #Updating objective function to reflect the change in costs.

In [43]:
opt3 = solver.solve(m3)


### 11)

In [44]:
for i in rows:
  for j in cols:
    num3 = m3.x[i,j]()
    if num3 != 0:
      print('Number of units transported from {} to {} is {}'.format(df2.index[i],df2.columns[j],num3))

Number of units transported from Ahmedabad to Jodhpur is 185.0
Number of units transported from Bengaluru to Vijayawada is 350.0
Number of units transported from Chennai to Trichy is 375.0
Number of units transported from Chennai to Vijayawada is 30.0
Number of units transported from Delhi to Jodhpur is 200.0
Number of units transported from Delhi to Simla is 70.0
Number of units transported from Delhi to Agartala is 10.0
Number of units transported from Kolkata to Dibrugarh is 100.0
Number of units transported from Kolkata to Raipur is 5.0
Number of units transported from Lucknow to Agartala is 75.0
Number of units transported from Mumbai to Hassan is 175.0
Number of units transported from Mumbai to Jodhpur is 15.0
Number of units transported from Mumbai to Trichy is 175.0
Number of units transported from Mumbai to Raipur is 60.0
Number of units transported from Hyderabad to Vijayawada is 270.0


In [45]:
print('Optimal total cost of transportation is',m3.cost())

Optimal total cost of transportation is 1289875.0


The optimal cost has increased. this is because Lucknow -Simla and Kolkata-Agartala routes has a nonzero flow in the original problem, and since these have to be zero now, a more costly combination of routes has to be chosen.